In [2]:
# Import packages
import plotly.graph_objs as go
import pandas as pd


In [3]:
arbeidsmarkt = pd.read_csv("../data/arbeidsmarkt.csv", sep=';')
uurloon = pd.read_csv("../data/uurloon.csv", sep=';')
uurloon['Uurloon werknemers na verlaten ho (euro)']  = pd.to_numeric(uurloon['Uurloon werknemers na verlaten ho (euro)'] , errors='coerce')
arbeidsmarkt['Uitstromers ho (aantal)']  = pd.to_numeric(arbeidsmarkt['Uitstromers ho (aantal)'] , errors='coerce')
arbeidsmarkt.fillna(0, inplace = True)

peilmoment1 = arbeidsmarkt.loc[arbeidsmarkt['Peilmoment'] == 'Direct na verlaten onderwijs']

verdeling = peilmoment1.groupby(['Geslacht','Studierichting'])['Uitstromers ho (aantal)'].sum().reset_index()
verdeling.rename(columns={'Uitstromers ho (aantal)': 'Total studierichting'}, inplace=True)

In [89]:
labels = []
parents = []
values = []

unique_geslacht = set()

label_map = {
    'Recht': 'Law',
    'Gedrag en Maatschappij': 'Behaviour & Society',
    'Gezondheidszorg': 'Healthcare',
    'Onderwijs': 'Education',
    'Economie': 'Economics',
    'Natuur': 'Nature',
    'Techniek': 'STEM',
    'Onderwijs': 'Education',
    'Sectoroverstijgend': 'Other',
    'Landbouw en natuurlijke omgeving': 'Agriculture',
    'Taal en cultuur': 'Language & Culture',
    'Vrouwen': 'Women',
    'Mannen': 'Men'
}

for _, row in verdeling.iterrows():
    geslacht = row['Geslacht']
    studierichting = row['Studierichting']
    aantal = row['Total studierichting']

    if label_map[geslacht] not in unique_geslacht:
        labels.append(label_map[geslacht])
        parents.append("")
        if geslacht == 'Vrouwen':
            values.append(44000.0)
        elif geslacht == 'Mannen':
            values.append(35170.0)
        unique_geslacht.add(label_map[geslacht])

    if geslacht == 'Mannen':
        labels.append(label_map[studierichting] + ' ♂')
    elif geslacht == 'Vrouwen':
        labels.append(label_map[studierichting] + ' ♀')
    parents.append(label_map[geslacht])
    values.append(aantal)

fig = go.Figure(go.Sunburst(
    labels=labels,
    parents=parents,
    values=values,
    branchvalues='total',
    insidetextfont=dict(size=20),
    hoverinfo='value+text',
    hovertext='graduates'
))

fig.update_layout(
    title='Study subject by sex<br><sup>Women favour Behaviour & Society and men STEM',
    margin=dict(t=50, l=25, r=25, b=25),
    height=600,
    margin_b=100,
    annotations=[
        go.layout.Annotation(
            text="The sunburst illustrates the distribution of fields of study among men and women.<br>The chart shows that the stereotypical gender distribution aligns partly with our data.",
            xref="paper", yref="paper",
            x=0, y=-0.15,
            showarrow=False,
            font=dict(size=12),
            xanchor='left',
            yanchor='bottom',
            align='left'
        ),
    ]
)
print(fig.data[0].hovertemplate)
fig.show()

None
